# Run Apache Beam on Azure Synpase

Install Apache Beam and dependencies

```
apache-beam[interactive]
pyarrow
pandas==1.0.0
```


In [ ]:
import apache_beam as beam

with beam.Pipeline() as pipeline:
  lines = (
      pipeline
      | beam.Create([
          'To be, or not to be: that is the question: ',
          "Whether 'tis nobler in the mind to suffer ",
          'The slings and arrows of outrageous fortune, ',
          'Or to take arms against a sea of troubles, ',
      ])
      )

## Update Azure Storage Connection string

In [ ]:
connection_string = ""
os.environ['AZURE_STORAGE_CONNECTION_STRING'] = connection_string

In [ ]:
import os
import re
import apache_beam as beam
from apache_beam.io.azure.blobstoragefilesystem import BlobStorageFileSystem


sampleText = 'azfs://hyundevsynapsestorage/dev-synapse/user/trusted-service-user/beam/sample/sample.txt'
outputPath = 'azfs://hyundevsynapsestorage/dev-synapse/user/trusted-service-user/beam/output/wordcount.txt'


with beam.Pipeline() as pipeline:
  lines = (
      pipeline
      | beam.io.textio.ReadFromText(sampleText)
      | beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | beam.Map(lambda word: (word, 1))
      | beam.CombinePerKey(sum)
      | beam.Map(lambda word_count: str(word_count))
      | beam.io.textio.WriteToText(outputPath)
      )


In [ ]:
outputPath = 'azfs://hyundevsynapsestorage/dev-synapse/user/trusted-service-user/beam/output/hamlet.txt'

with beam.Pipeline() as pipeline:
  lines = (
      pipeline
      | beam.Create([
          'To be, or not to be: that is the question: ',
          "Whether 'tis nobler in the mind to suffer ",
          'The slings and arrows of outrageous fortune, ',
          'Or to take arms against a sea of troubles, ',
      ])
      | beam.io.textio.WriteToText(outputPath)
      )


In [ ]:
# sampledata_path = 'azfs://hyundevsynapsestorage/dev-synapse/data/bingCOVID19/*'

# with beam.Pipeline() as pipeline:
#   lines = (
#       pipeline
#       | beam.io.parquetio.ReadFromParquet(sampledata_path)
#       | beam.Map(lambda table: table.to_pandas())
#       )

In [ ]:
import pyarrow 

outputdata_path = 'azfs://hyundevsynapsestorage/dev-synapse/data/beamoutput/outputsample.parquet'
with beam.Pipeline() as p:
  records = p | 'Read' >> beam.Create(
      [{'name': 'foo', 'age': 10}, {'name': 'bar', 'age': 20}]
  )
  _ = records | 'Write' >> beam.io.WriteToParquet(outputdata_path,
      pyarrow.schema(
          [('name', pyarrow.binary()), ('age', pyarrow.int64())]
      )
  )

In [ ]:
%%pyspark
df = spark.read.load([
    'abfss://dev-synapse@hyundevsynapsestorage.dfs.core.windows.net/data/beamoutput/outputsample.parquet-00000-of-00001'
    ], format='parquet')
display(df.head(2))